In [1]:
import numpy as np

In [2]:
from nn_labs.nn.layers import BaseLayer
from nn_labs.nn.activation_functions import BaseActivationFunction

from numpy.typing import NDArray

class Network:
    def __init__(self):
        self.layers: list[BaseLayer | BaseActivationFunction] = []
    
    def add_layer(self, l: BaseLayer | BaseActivationFunction):
        self.layers.append(l)
        
    def forward(self, x_in: NDArray) -> NDArray:
        current = x_in
        
        for layer in self.layers:
            current = layer.forward(current)
            
        return current
    
    def backward(self, d_loss: NDArray):
        current_gradient = d_loss
        
        for layer in reversed(self.layers):
            print(current_gradient.shape, layer.__class__)
            layer.backward(current_gradient)
            
            current_gradient = layer.d_inputs

In [3]:
from nn_labs.nn.layers import Conv2D, DenseLayer, Flatten, MaxPool2D
from nn_labs.nn.activation_functions import ReLU, Sigmoid

bob_net = Network()

bob_net.add_layer(Conv2D(1, 2, 16, 1))
bob_net.add_layer(MaxPool2D(2, 2))
bob_net.add_layer(Conv2D(16, 2, 16, 1))
bob_net.add_layer(MaxPool2D(2, 2))
bob_net.add_layer(Flatten())
bob_net.add_layer(DenseLayer(256, 400))
bob_net.add_layer(ReLU())
bob_net.add_layer(DenseLayer(400, 784))
bob_net.add_layer(Sigmoid())

In [4]:
x = np.ones((100, 1, 28, 28))

out = bob_net.forward(x)

out.shape

(100, 784)

In [5]:
from nn_labs.nn.perf import MSELoss
from nn_labs.image_net.load import load_fashion_mnist

(X_train, Y_train), (X_test, Y_test) = load_fashion_mnist()

X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

EPOCHS = 2000
BATCH_SIZE = 4086
LEARNING_RATE = 10e-3
batch_count = X_train.shape[0] // BATCH_SIZE

loss_fn = MSELoss()

for epoch in range(EPOCHS):
    print(f"Epoch: {epoch}")
    for batch_no in range(batch_count):
        start_idx = batch_no * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE

        x = X_train[start_idx:end_idx]
        y = Y_train[start_idx:end_idx]
        
        x_cmp = x.reshape(BATCH_SIZE, 784)
        
        prediction = bob_net.forward(x)
        loss = loss_fn.calculate(prediction, x_cmp)
        
        loss_fn.backward(prediction, x_cmp)
        bob_net.backward(loss_fn.d_inputs)


Epoch: 0
(4086, 784) <class 'nn_labs.nn.activation_functions.Sigmoid'>
(4086, 784) <class 'nn_labs.nn.layers.DenseLayer'>
(4086, 400) <class 'nn_labs.nn.activation_functions.ReLU'>
(4086, 400) <class 'nn_labs.nn.layers.DenseLayer'>
(4086, 256) <class 'nn_labs.nn.layers.Flatten'>
(4086, 16, 4, 4) <class 'nn_labs.nn.layers.MaxPool2D'>
(4086, 16, 10, 10) <class 'nn_labs.nn.layers.Conv2D'>
(4086, 16, 12, 12, 2, 2)
(16, 16, 2, 2)
(4086, 12, 12, 16)
---------------------------------------------------------------------
(4086, 12, 12, 16) <class 'nn_labs.nn.layers.MaxPool2D'>


IndexError: index 12 is out of bounds for axis 1 with size 12